In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    "data/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
    subset="training"
)
val_generator = train_datagen.flow_from_directory(
    "data/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

In [ ]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(150, 150, 3),
    include_top=False,
    weights="imagenet"
)

# Freeze base model
base_model.trainable = False

# Add custom classifier
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(train_generator, validation_data=val_generator, epochs=5)

In [ ]:
acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
print(f"Train Accuracy: {acc:.2f}, Validation Accuracy: {val_acc:.2f}")

model.save("models/cats_dogs_resnet.h5")